In [1]:
#r "nuget: Fli"

open Fli
open System.IO

let postsFlaDir = "/Users/flavioc/Web/fcolavecchia/_posts/fsharp"
let srcDir = "/Users/flavioc/Documents/Blog/"



Installed Packages Fli, 1.0.1

In [8]:
let nbConvert file =
    let fullName = Path.Combine(srcDir,"en",file)
    $"/opt/miniconda3/bin/jupyter nbconvert --to markdown {fullName}.ipynb --no-prompt"

In [18]:
let cliConvert inFile outfile = 
    cli {
        Exec (nbConvert inFile)
        // Arguments $"--output={outfile}" 
        WorkingDirectory (Path.Combine(srcDir,"tools"))
    }

let dry (cliValue:  ExecContext) =    
    cliValue 
    |> Command.toString

let run (cliValue:  ExecContext) =    
    cliValue 
    |> Command.execute


In [11]:
let allNotebooks = Directory.GetFiles(Path.Combine(srcDir,"en/"),"*.ipynb")
let notebooks = 
    [
        "Fundamentals"
        // "Functions"
        // "IntroToTypes"
        // "DiscriminatedUnions"
    ]
    
notebooks

index,value
0,Fundamentals


In [5]:
let posts = 
    notebooks
    |> List.mapi (fun i n -> string i + "_"+n)
    |> List.map (fun n -> Path.Combine(postsFlaDir,n+".md"))
    // |> List.map (fun (ni,no) -> dry (cliConvert ni no))
posts

index,value
0,/Users/flavioc/Web/fcolavecchia/_posts/fsharp/0_Fundamentals.md


In [19]:
(notebooks, posts)
    ||> List.map2 (fun ni no -> dry (cliConvert ni no))

index,value
0,/opt/miniconda3/bin/jupyter nbconvert --to markdown /Users/flavioc/Documents/Blog/en/Fundamentals.ipynb --no-prompt


In [20]:
(notebooks, posts)
    ||> List.map2 (fun ni no -> run (cliConvert ni no))

Error: System.ComponentModel.Win32Exception (2): An error occurred trying to start process '/opt/miniconda3/bin/jupyter nbconvert --to markdown /Users/flavioc/Documents/Blog/en/Fundamentals.ipynb --no-prompt' with working directory '/Users/flavioc/Documents/Blog/tools'. No such file or directory
   at System.Diagnostics.Process.ForkAndExecProcess(ProcessStartInfo startInfo, String resolvedFilename, String[] argv, String[] envp, String cwd, Boolean setCredentials, UInt32 userId, UInt32 groupId, UInt32[] groups, Int32& stdinFd, Int32& stdoutFd, Int32& stderrFd, Boolean usesTerminal, Boolean throwOnNoExec)
   at System.Diagnostics.Process.StartCore(ProcessStartInfo startInfo)
   at System.Diagnostics.Process.Start(ProcessStartInfo startInfo)
   at Fli.Command.startProcess(FSharpFunc`2 inputFunc, FSharpFunc`2 outputFunc, ProcessStartInfo psi) in C:\Users\const\github\repos\Fli\src\Fli\Command.fs:line 67
   at Fli.Command.Command.execute(ExecContext context) in C:\Users\const\github\repos\F